# Import modules

In [ ]:
import numpy as np
import pandas as pd
import os
%matplotlib notebook
%load_ext autoreload
%autoreload 2
import matplotlib as mpl
import matplotlib.pyplot as plt
from datetime import datetime
from datetime import timedelta
from IPython.display import clear_output
os.chdir(os.environ['HOME'] + '/Google Drive/Research/Project_LandSeaContrast/')
print(os.getcwd())
import myfuncs as my
from IPython.display import display
    # for displaying pandas tables
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

# Sorting test

In [ ]:
print(np.arange(5).shape, np.random.rand(5).shape)
np.interp((0.5,1.5,2.5), range(5),a)

In [ ]:
help(np.interp)

# Vector matching tools

In [ ]:
from importlib import reload
reload(my)
a = pd.DataFrame({'a':[1,2],'b':[3,4]},
                 index=[datetime(1970,1,1),datetime(1970,2,1)])
b = pd.DataFrame({'a':[1,2,3],'b':[4,5,6]},index=[datetime(1970,1,1),datetime(1970,2,1),datetime(1970,3,1)])
idfilt, filt = my.match(a.index,b.index)
print('Loc:',a.loc[a.index[filt[0]]])
print('iLoc:',a.iloc[1:2,:])
print('iLoc, converting to Series:',a.iloc[1,:])
try:
    print('Simple index:',a[idfilt])
except:
    print('Loc index with ids:',a.loc[idfilt])
    print('Must use .loc to filter with IndexType.')
print('Index month:',a.index[0].month)
print('Index type:',type(a.index))
print('Sum, converting to Series:',b.sum(0))
print('Values:', a.index.values, type(a.index.values[0]), type(a.index.values))

# Datetime stuff

In [ ]:
x = [datetime(y,1,1) for y in range(2000,2010)]
y = np.datetime64(x)

# Pandas Dataframe Stuff

## Grouping with datetime indices

In [ ]:
# integer grouping
d = pd.DataFrame(dict(col1=[2,1,3,1,0]))
d.index//3
    # just like array division; then groups rows with common indices into
    # this abstract "GroupBy" object onto which you can apply operations!
f = d.groupby(d.index // 3).mean()
display(f)
# datetime grouping
c = pd.DataFrame(dict(a=[1,2,3,4]), 
                 index=[datetime(1970,1,1), datetime(1970,2,1), 
                        datetime(1971,1,1), datetime(1971,2,1)])
display(c)
cy = c.groupby(c.index.year).mean()
print('Yearly mean:')
display(cy)
cm = c.groupby(c.index.month).mean()
print('Monthly mean:')
display(cm)
print('January filter:')
mfilt = c.loc[c.index.month==1]
display(mfilt)
# or try mask
print(c.index.month)
np.in1d(c.index.month,[1,12])

## Seasonal means with groupby

In [ ]:
# and try seasonal means using groupby
c = pd.DataFrame(dict(a=range(24)), 
                 index=[datetime(1970,i,1) for i in range(1,13)] + 
                [datetime(1971,i,1) for i in range(1,13)])
season_mean = c.groupby((c.index.year + (c.index.month // 12), c.index.month % 12 // 3)).mean()
display('season mean:',season_mean)
display('index level 1:',season_mean.index.levels[0])
try:
    season_mean.groupby(season_mean.index.levels[0]).mean()
except:
    print('Grouping by with index reference does not work, because'
          + 'pandas returns only the unique ids in order!')
# instead use groupby option
display('Annual mean:',season_mean.groupby(level=0).mean())
display('')

## Building a MultiIndex from scratch/suppressing new one in groupby

In [ ]:
# multiindexing, and applying inline running mean
M = pd.MultiIndex.from_product((range(1961,1991), range(1,13)))
m = pd.DataFrame({'a':range(30*12)}, index=M)
rollmean = m.groupby(level=1, group_keys=False, squeeze=True).rolling(window=5,center=True).mean()
display(rollmean)
# next, subtract result
resid = m - rollmean

## Changing Multi-Index entries 

In [ ]:
# reset a multiindex
M = pd.MultiIndex.from_product((range(1961,1991), range(1,13)))
m = pd.DataFrame({'a':range(30*12)}, index=M)
print('Reindex does not work for this; result:',m.reindex(index=range(30*12), level=0)) # doesn't work; not intended useage
try: 
    m.index.levels[0] = range(30*12)
except:
    print('Cannot set levels by assignment, since they are not mutable.')
print('Instead, use set_levels with "level" kwarg.')
m.index = m.index.set_levels(range(30), level=0)
display('With new index:',m)
# and what about iterating over objects? can we do it?
for i in (m,):
    i.index = i.index.set_levels(range(30,60), level=0)
display('After list modification:',m)
display(type(m.a))

## Modifying objects in list, and the reference shows modified object

In [ ]:
# modifying a bunch of ndarrays, for example
a = np.arange(5,11)
b = np.arange(6)
for i in [a, b]:
    i -= 1
a, b

## Grouping dataframes 

In [ ]:
# reset a multiindex
M = pd.MultiIndex.from_product((range(1961,1991), range(1,13)))
m = pd.DataFrame({'a':range(30*12)}, index=M)
p = pd.Panel({'Had':m, 'CRUT':0 - m})
# display(p.Had, p.CRUT)
# keep separate dataframes, but apply to list? kinda slow
# %timeit (i.groupby(level=1, group_keys=False, squeeze=True).rolling(window=5,center=True).mean() for i in [p.Had, p.CRUT])

# make panel? kind of slow; assignment is big one, more than extra layer of indexing
#%timeit (p[i].groupby(level=1, group_keys=False, squeeze=True).rolling(window=5,center=True).mean() for i in p)
display(p.Had.loc[1975])
%timeit for i in p: p[i] = p[i].groupby(level=1, group_keys=False, squeeze=True).rolling(window=5,center=True).mean()
for i in p: p[i] += p[i].groupby(level=1, group_keys=False, squeeze=True).rolling(window=5,center=True).mean()
display(p.Had.loc[1975])

# what if we concatenate two dataframes, instead?
c = pd.concat((m, 1-m), axis=1, keys=('Had', 'CRUT'))
display(c)
%timeit for i in [c]: i += i.groupby(level=1, group_keys=False, squeeze=True).rolling(window=5,center=True).mean()
c += c.groupby(level=1, group_keys=False, squeeze=True).rolling(window=5,center=True).mean()

#p.apply(lambda x: x.index, axis=('major','minor'))

## Dot-referencing DataFrame columns 

In [ ]:
# referencing object with multi columns
c = pd.concat((m, 1-m), axis=1, keys=('Had', 'CRUT'))
# print(c.loc[1970,1], c.Had.a)
# and creating new ones with dots!
# display(c)
for i in c.columns.levels[0]:
    c[i, 'b'] = 2*m
display(c.columns)

## Seasonal mean/working with MultiIndex month/year data

In [ ]:
N = 12*2
c = [datetime(1970 + i//12, (i%12)+1, 1) for i in range(N)]
z = pd.Index(c)
print(z)
d = pd.DataFrame(np.random.rand(N), index=c)
print(d)
# attempt 0
m=pd.MultiIndex.from_arrays((d.index.year, d.index.month), names=('year','month'))
print(m)
q = d.copy()
q.index = m
print(q)
# new attempt
a = np.array(c)
s = pd.DataFrame(index=a, names)
# attempt 1
# d = d.groupby((d.index.year, d.index.month)).sum()
# print(n)
# d.index = d.index.set_names(['year', 'month'])
# print(d)
# d.index.names = ['year', 'month']
# print(d)
# # what about starting an empty one?
# d = pd.DataFrame(index=c).groupby((c.year, c.month)).sum()


## Concatenating/adding to dataframe with named vs. unnamed indices of identical value 

### Evidently passing index as TUPLE VS. NON-TUPLE MATTERS!
Perhaps tuples are passed as convenience for some other things, but if you want to build up implicit MultiIndex, must be (1) list of (2) identical-length iterables.

In [ ]:
a = pd.DataFrame(dict(a=range(3)), index=[[0,0,0],[1,2,3]])
display(a)
b = pd.Index([datetime(1970,1,1), datetime(1970,1,2), datetime(1970,1,3)])
a = pd.DataFrame(dict(a=range(3)), index=[b.year, b.day])
display(a)
    # ...actually I'm wrong, must be list or ndarray... but b.year, b.day work because they 
    # return ndarray, because pandas works HEAVILY/CLOSELY with numpy
b.year
# assignment time differences: Series vs. stuff
N = 12*10
c = pd.Index([datetime(1970 + i//12, (i%12)+1, 1) for i in range(N)])
m = pd.MultiIndex.from_arrays([c.year, c.month], names=('year','month'))
d = pd.DataFrame()
# %timeit d['a'] = pd.Series(np.arange(N), index=m)
# %timeit d['a'] = np.arange(N)
d['a'] = pd.Series(np.arange(N), index=m)
# assigning with names vs without
q = pd.Series({'b':1}, index=[[1970],[1]])
display(d)
# %timeit pd.concat((q,d), axis='columns')
# %timeit pd.concat((q,d), axis='columns', copy=False)

d['b'] = pd.Series([1], index=[[1970],[1]])
display(d)
d['c'] = pd.DataFrame([1], index=[[1970], [1]])
d


## Subtracting monthly means

In [ ]:
N = 12*30
c = pd.Index([datetime(1970 + i//12, (i%12)+1, 1) for i in range(N)])
d = pd.DataFrame(np.random.rand(N), index=[c.year, c.month])
d.index.names = ('year', 'month')
base = d.loc[1980:1990].groupby(level='month').mean()
print('Table of monthly means in base period:')
display(base)
print('Then subtract the table; what happens?')
display(d.iloc[:10,:])
display((d-base).iloc[:10,:])
season_tuple = (c.year + (c.month // 12), (c.month % 12) // 3)
d.groupby(season_tuple,group_keys=[2,3,4,5]).mean()

## Creating new columns with partially overlapping rows

In [ ]:
# Assigning new data to ONLY CERTAIN ROWS in dataframe
N = 12*10
index = pd.Index([datetime(1970 + i//12, (i%12)+1, 1) for i in range(N)])
d = pd.DataFrame(dict(a=np.random.rand(N)), index=pd.MultiIndex.from_arrays((index.year, index.month)))
d.loc[[1970, 1972]] += 5
d2 = pd.DataFrame(dict(simple_concat=1), index=[[1970], [1]])
d = pd.concat((d, d2),axis=1,join='outer')
# Or what about assigning a series? can't make new indices inline, evidently
try:
    d['from_series'] = pd.Series([1], index=[[1970],[13]])
    print(d.loc[1970,13])
except:
    print('Cannot add new index by Series assignment; only be concatenation, it seems.')
    d3 = pd.DataFrame(dict(newrow_concat=[1]), index=[[1970],[13]])
    d3.name = ('2','new')
    d = pd.concat((d,d3),axis='columns',keys=('a','b'))
    # or with join? to handle MultiIndices?
#     d.join(d3, on='outer', how='inner')
#     dipslay(d)
    # print(pd.concat((d,d3),axis=1)) # this FAILS TO WRITE NEW VALUE; 
    # so .concat can add new column implicitly, but CANNOT add new row implicitly
# And adding new series to MultiIndexed thing
s = pd.Series(dict(b=[1]),index=[[1970],[2]])
s.name = 'b'
# d = pd.concat((d, s), axis='columns', join='outer')
# display(d)
# Times
#%timeit d[2,'from_series'] = pd.Series([1], index=[[1970],[13]])
#%timeit f = pd.concat((d,d2),axis='columns')
# Or, if we concatenate DataFrames with join=oute
s = pd.DataFrame(dict(a=range(4)), index=[[0,1,2,3],[0,0,1,1]])
s

In [ ]:
# Setup
N = 12*10
index = pd.Index([datetime(1970 + i//12, (i%12)+1, 1) for i in range(N)])
d = pd.DataFrame(dict(a=np.random.rand(N)), index=pd.MultiIndex.from_arrays((index.year, index.month)))
pd.MultiIndex.from_product((range(1970,1972),range(1,13))).levels
d3
d2 = pd.DataFrame(dict(simple_concat=1), index=[[1970], [1]])
d = pd.concat((d, d2),axis=1,join='outer')
d3 = pd.DataFrame(dict(newrow_concat=[1]), index=[[1970],[13]])
d = pd.concat((d,d3),axis='columns',keys=('a','b'))
#n1 = pd.concat({**d, ('q','1'):pd.Series([1],index=[[1970],[1]])},axis='columns')
# display(dnew)
# print({('c',x):dnew[x] for x in dnew})
# a = {**d}.update({('c',x):dnew[x] for x in dnew})
# display(a)
# dict
dnew = pd.DataFrame(dict(a=[1]),index=[[1970],[1]])
print({**d})
%timeit pd.concat({**d, **{('c',nm):val for nm,val in dnew.items()}}, axis='columns')
q = pd.concat({**d, **{('c',nm):val for nm,val in dnew.items()}}, axis='columns')
display(q)
# s = pd.Series([1],index=[[1970],[1]])
# d['c','x'] = pd.Series(s)
# display(d)
# d['d'] = pd.DataFrame({'a':[1]},index=[[1970],[1]])
# display(d)
# %timeit d['c','x'] = pd.Series(s)
# # n2 = pd.concat({**d}.update({('c',x):dnew[x] for x in dnew}))

In [ ]:
import scipy.stats as st
print(d.index.get_level_values(0))
print(d['a','a'].values.ravel())
print(d.shape)
print(list(zip(*d.index.values))[0])
st.linregress(list(zip(*d.index.values))[0],d['a','a'].values)
d.groupby(level=0).rolling(window=5).apply(lambda x: st.linregress(range(x.size),x)[0])

In [ ]:
# some math
omega = 7.2921159e-5
R = 6371.0e3
beta = lambda lat: 2*omega*np.cos(lat*np.pi/180)/R
cor = lambda lat: 2*omega*np.sin(lat*np.pi/180)
print(beta(30), beta(50))
print(cor(30), cor(50))
# distance
print('%.3e' % (R*10*np.pi/180))
# quadratic interpolation
import scipy.interpolate as pol
i = [0, 5, 10] # distances in latitude
samp = [40, 45, 40]
samp = [25, 20, 17.5]
f = np.polyfit([j*R*np.pi/180 for j in i],samp, 2)
f
# scale height
287*220/9.80665
np.log(1000/200) - np.log(1000/500)
Zdiff = lambda a,b: np.log(1000/a) - np.log(1000/b)
print(Zdiff(300,400))
# change
1.0e6


## Rolling mean on groupby object

In [ ]:
N = 12*60
index = pd.Index([datetime(1970 + i//12, (i%12)+1, 1) for i in range(N)])
d = pd.DataFrame(dict(a=np.random.rand(N)), index=pd.MultiIndex.from_arrays((index.year, index.month),names=('year','month')))
f = d.groupby(level=1,group_keys=False).rolling(window=3,center=True).mean()
f.sortlevel(level='year')
    # use this to re-sort by a level

In [ ]:
a = {'a':1, 'b':5}
a['a'] *= 2
a

# Misc testing

In [ ]:
# getting series or dataframe
a = pd.DataFrame({'one':[1,2,3], 'two':[4,5,6]})
a.index = [1,2,1]
display('Indexing for series vs. singleton dataframe:',type(a['one']))
display(type(a[['one']]))
d.iloc[1::12]
s = pd.Series(range(12))
s[::2]
d
d.index.levels[d.index.names.index('year')]

In [ ]:
a, b = np.meshgrid(range(5),range(5,10))
a, b = a.ravel(), b.ravel()
a, b
d.index.levels[0][d.index.labels[0]]
d.index.get_level_values(1)
# import matplotlib.pyplot as plt
plt.plot(d.index.get_level_values('year'),d.a)
plt.show()
d.index.get_level_values(0)[::12]

In [ ]:
# meshgrid behavior -- using lon as arg1, lat as arg2 puts lons on 
# ROWS, lats on COLUMNS (because row-major)... and this is what we 
# actually want for coordinates
from mpl_toolkits.basemap import Basemap
m = Basemap(projection='moll', lon_0=0, resolution='c')
lon = np.arange(-180,181,60)
lat = np.arange(-90,91,30)
lon2d, lat2d = np.meshgrid(lon, lat)
x, y = m(lon2d, lat2d)
lon2d, lat2d
# slice(1,5).indices()
np.array([1,2,3])[range(2)]
[i for i in slice(5)]


In [ ]:
# copying dataframe objects; must use .copy()
d1 = pd.DataFrame({'a':[1]})
d2 = d1.copy()
d1['a'] = 2
display('With copy:',d1)
display(d2)
# no copy
d1 = pd.DataFrame({'a':[1]})
d2 = d1
d1['a'] = 2
display('Without copy:',d1)
display(d2)

In [ ]:
# pcolor: provide color polygon boundaries, and data to fill them
# also remember python is row-major; so rows of visual array match x
a = np.random.rand(5,5)
print(a)
print(a.ravel())
fig, ax = plt.subplots()
ax.pcolor(np.array([1,2,3,4]),np.array([1,2,3]), 
           np.random.rand(2,3))
plt.show()

# Utils testing

In [ ]:
help(fig.tight_layout)
fig, ax = plt.subplots(2,2)
fig.tight_layout()
plt.show()
help(fig.colorbar)

In [ ]:
def completeyears(dt, startmonth=1):
    """
    Input...
        dt (positional, required): an ndarray of datetime objects
        startmonth (kw, optional): the starting month characterizing a "complete" 
            year (1 is default, but 12 is also common)
    Returns...
     1) list of filtered datetimes
     2) boolean ndarray vector that can be used to filter out complete years
        given some ndarray of ***monthly*** observation datetimes.
    """
    # some basic stuff
    vfunc = np.vectorize(lambda dt: dt.month)
    endmonth = ((startmonth - 2) % 12) + 1
    # get filter
    print(startmonth, endmonth, vfunc(dt))
    m_idstart = np.where(vfunc(dt)==startmonth)[0][0]
    m_idend = np.where(vfunc(dt)==endmonth)[0][-1]
    filt = np.arange(m_idstart, m_idend+1)
    
    return dt[filt], filt
        # returns only up to the last instance of last month


In [ ]:
# Decomposing output
def f():
    return 1, (2, 3)
_, (_, b) = f()
print(b)

In [ ]:
# Splitting stuff into pieces
a = np.random.randn(3,3)
print(a)
b = np.split(a, (1,2), axis=0)
    # can use integer for equal division, or tuple/list
    # division points (e.g. 1 splits 0 and 1:)
for i,b in enumerate(b):
    print(i, b)

In [ ]:
# np.where behavior
np.where(np.array([1,2,3])==4)[0]
print(not np.array([]))
print(not np.array([0]))
for a in np.array([1,2,3]):
    print(type(a))

In [ ]:
list(globals().keys())

In [ ]:
# Generate monthly datetime sequence
a = np.array([datetime(2000,5,1)])
for i in range(1,48):
    a = np.append(a,datetime(a[-1].year + a[-1].month//12, (a[-1].month % 12)+1, a[-1].day))

In [ ]:
print(completeyears(a, startmonth=5))

In [ ]:
def mesh(lon, lat):
    '''
    Gets graticule for full globe of longitude, latitude data.
    '''
    # fix lons
    lodiff = lon[1]-lon[0]
    hidiff = lon[-1]-lon[-2]
    lon = np.concatenate((
            np.array([lon[0]-lodiff/2]), (lon[1:]+lon[:-1])/2, np.array([lon[-1]+hidiff/2])
            ))
    # fix lats
    lat = np.concatenate((
            np.array([-90]), (lat[1:]+lat[:-1])/2, np.array([90])
            ))
    return lon, lat

testlon = np.arange(-180., 180., 30.)
testlat = np.arange(-85., 85.1, 10.)
mesh(testlon, testlat)

# Pandas testing

In [ ]:
import pandas as pd
import os
d = os.getcwd()
d = d + '/Google Drive/Research/Project_LandSeaContrast'
os.listdir(d)
eri = pd.read_table(d + '/eri.txt',usecols=(5,6,7),skiprows=9,header=None,delim_whitespace=True) #, usecols=(6,7,8))
eri.values
#eri

# Graphics testing

In [ ]:
# do new graphics reset the axes title, labels, etc?
import matplotlib.pyplot as plt
a = np.random.rand(5,5)
fig, ax = plt.subplots()
ax.plot(a)
plt.title('abcef')
ax.plot(a)
plt.show()
fig, ax = plt.subplots()
p=ax.pcolor(a,cmap='Greys',clim=(0,0.5))
fig.colorbar(p)